In [1]:
import numpy as np

import json
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
import keras.backend as K

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = np.load('data/sentipolc_char_seq.npz')

In [2]:
with open('data/sentipolc_char_index.json', 'r') as f:
    word_index = json.load(f)

In [3]:
with open('data/chars.tsv', 'w') as f:
    for w,_ in word_index.items():
        f.write('{}\n'.format(w))

In [4]:
words = [w for w,_ in word_index.items()]

In [5]:
data = np.load('data/sentipolc_char_seq.npz')

x_train = data['x_train']
y_train = data['y_train']

x_test = data['x_test']
y_test = data['y_test']

In [6]:
import pandas as pd
pd.Series(y_train).value_counts()

0    5100
1    2310
dtype: int64

Check the max length of the text

In [7]:
max_len_seq = max([len(x) for x in x_train])
print('max len seq {}'.format(max_len_seq))
max_idx = max(word_index.values())
print('max id {}'.format(max_idx))

max len seq 238
max id 137


In [8]:
x_train_pad = sequence.pad_sequences(x_train, maxlen=max_len_seq, padding='post')
x_test_pad = sequence.pad_sequences(x_test, maxlen=max_len_seq, padding='post')

In [9]:
x_train_pad.shape

(7410, 238)

# Define metrics to use

In [10]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.
     Computes the precision, a metric for multi-label classification of
     how many selected items are relevant.
      """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))		
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))		
    precision = true_positives / (predicted_positives + K.epsilon())		
    return precision

In [11]:
def recall(y_true, y_pred):
    """Recall metric.
 
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

split_train_test = StratifiedShuffleSplit(1,test_size=0.2, random_state=123456)

for train, test in split_train_test.split(x_train_pad, y_train):
    x_train_pad_split, y_train_pad_split = x_train_pad[train], y_train[train]
    x_val_pad_split, y_val_pad_split = x_train_pad[test], y_train[test]

In [13]:
callbacks = [
    TensorBoard(histogram_freq=1, batch_size=128, embeddings_freq=1),
    ReduceLROnPlateau(patience=2, verbose=1)
]

Instructions for updating:
Use the retry module or similar alternatives.


In [14]:
def build_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_idx+1, output_dim=100, input_length=max_len_seq))
    model.add(Conv1D(filters=64, kernel_size=5, padding='same', activation='relu', strides=1))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu', strides=1))
    model.add(GlobalMaxPool1D())
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy', recall, precision])
    
    return model

In [15]:
model = build_model()
model.summary()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 238, 100)          13800     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 238, 64)           32064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 119, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 119, 32)           10272     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                1650      
___________________________________________________________

In [16]:
history = model.fit(x_train_pad_split, y_train_pad_split, validation_data=(x_val_pad_split, y_val_pad_split),
                    batch_size=128, validation_split=0.2, epochs=10, callbacks=callbacks)

Train on 5928 samples, validate on 1482 samples
Epoch 1/10
5928/5928 [==============================] - 10s 2ms/step - loss: 0.6343 - acc: 0.6834 - recall: 0.0173 - precision: 0.0571 - val_loss: 0.6130 - val_acc: 0.6883 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/10
5928/5928 [==============================] - 11s 2ms/step - loss: 0.6111 - acc: 0.6910 - recall: 0.0209 - precision: 0.3388 - val_loss: 0.5952 - val_acc: 0.6997 - val_recall: 0.0722 - val_precision: 0.6693
Epoch 3/10
5928/5928 [==============================] - 11s 2ms/step - loss: 0.5881 - acc: 0.7080 - recall: 0.1205 - precision: 0.6933 - val_loss: 0.5768 - val_acc: 0.7085 - val_recall: 0.1542 - val_precision: 0.6372
Epoch 4/10
5928/5928 [==============================] - 11s 2ms/step - loss: 0.5683 - acc: 0.7198 - recall: 0.2111 - precision: 0.6658 - val_loss: 0.5691 - val_acc: 0.7240 - val_recall: 0.2641 - val_precision: 0.6365
Epoch 5/10
5928/5928 [==============================] - 10s 2ms/step - loss:

## Evaluate on the test set

In [17]:
callbacks = [
    TensorBoard(histogram_freq=0, batch_size=128, embeddings_freq=1),
]

In [18]:
model = build_model()

history = model.fit(x_train_pad, y_train, batch_size=128, epochs=6, callbacks=callbacks)

Epoch 1/6
7410/7410 [==============================] - 11s 1ms/step - loss: 0.6270 - acc: 0.6856 - recall: 0.0054 - precision: 0.0243
Epoch 2/6
7410/7410 [==============================] - 11s 1ms/step - loss: 0.6070 - acc: 0.6883 - recall: 0.0000e+00 - precision: 0.0000e+00
Epoch 3/6
7410/7410 [==============================] - 11s 1ms/step - loss: 0.5996 - acc: 0.6885 - recall: 0.0043 - precision: 0.1382
Epoch 4/6
7410/7410 [==============================] - 11s 1ms/step - loss: 0.5865 - acc: 0.6970 - recall: 0.0527 - precision: 0.4738
Epoch 5/6
7410/7410 [==============================] - 11s 1ms/step - loss: 0.5691 - acc: 0.7188 - recall: 0.2087 - precision: 0.6851
Epoch 6/6
7410/7410 [==============================] - 11s 1ms/step - loss: 0.5493 - acc: 0.7333 - recall: 0.2977 - precision: 0.6722


In [19]:
model.evaluate(x_test_pad, y_test)

2000/2000 [==============================] - 2s 781us/step


[0.483341329574585, 0.7975, 0.16947090888023378, 0.39819166421890256]

# Overall metrics 

In [20]:
from sklearn import metrics

In [21]:
predictions = model.predict_classes(x_train_pad, batch_size=128)
tn, fp, fn, tp = metrics.confusion_matrix(y_train, predictions).ravel()

recall = tp/(tp+fp)
precision = tp/(tp+fn)
f1 = metrics.f1_score(y_train,predictions)

print ('precision: {}'.format(precision), '\nrecall: {}'.format(recall), '\nf1: {}'.format(f1))

precision: 0.3385281385281385 
recall: 0.7083333333333334 
f1: 0.4581136496777973


In [22]:
#model.save('models/sentipolc_cnn_char.hdf5')